Optimierungsprogramm für den effizientesten Weg für den Transport von Gütern zwischen verschiedenen Standorten mit der Interior Point Methode 

In [141]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

Es gibt die 4 Standorte, die alle durch Transportrouten miteinander Verbunden sind

In [142]:
locations = ['A', 'B', 'C', 'D']


Die gegebene Kostenmatrix repräsentiert die Kosten für den Transport von Gütern zwischen verschiedenen Standorten. Die Matrix ist eine quadratische Matrix, bei der jedes Element (i, j) den Kostenwert für den Transport von Standort i zum Standort j angibt.

In [143]:
# Kostenmatrix der Wege
costs = [[0, 1, 2, 40],
         [1, 0, 21, 2],
         [2, 21, 0, 34],
         [40, 2, 34, 0]]



In [144]:
# Nachfrage zwischen den Standorten
demand = [150, 50, 200, 300]

# Kapazitäten der Transportwege
capacities = {'A': 110, 'B': 100, 'C': 300, 'D': 200}

# Anzahl der Standorte
num_locations = len(locations)

Formulierung des Optimierungsproblems:

In [145]:
# Gurobi-Modell erstellen
m = gp.Model('Transportation')

# Entscheidungsvariablen
x = m.addVars(num_locations, num_locations, vtype=GRB.CONTINUOUS, lb=0, name='x')

# Zielfunktion: Minimierung der Gesamtkosten
m.setObjective(gp.quicksum(costs[i][j] * x[i, j] for i in range(num_locations) for j in range(num_locations)), GRB.MINIMIZE)

Constraints bestehend aus Nachfragebedingungen und Kapazitätsbedingungen

In [146]:
# Nachfragebeschränkungen
m.addConstrs(gp.quicksum(x[i, j] for j in range(num_locations)) == demand[i] for i in range(num_locations))


# Kapazitätsbeschränkungen
m.addConstrs(gp.quicksum(x[i, j] for i in range(num_locations)) <= capacities[locations[j]] for j in range(num_locations))

# Transportwege von Standort A zu A, B zu B, C zu C und D zu D ausschließen
m.addConstr(x[0, 0] == 0)
m.addConstr(x[1, 1] == 0)
m.addConstr(x[2, 2] == 0)
m.addConstr(x[3, 3] == 0)

<gurobi.Constr *Awaiting Model Update*>

Ergebnis

In [147]:
# Modell optimieren
m.optimize()

# Ergebnisse ausgeben
if m.status == GRB.OPTIMAL:
    print('Optimal solution found:')
    for i in range(num_locations):
        for j in range(num_locations):
            print(f'x[{locations[i]}][{locations[j]}] = {x[i, j].x}')
else:
    print('No feasible solution found.')

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 16 columns and 36 nonzeros
Model fingerprint: 0xb26d78fc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 3e+02]
Presolve removed 5 rows and 5 columns
Presolve time: 0.02s
Presolved: 7 rows, 11 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5967680e+03   5.373613e+01   0.000000e+00      0s
       8    1.2580000e+04   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.258000000e+04
Optimal solution found:
x[A][A] = 0.0
x[A][B] = 0.0
x[A][C] = 100.0
x[A][D] = 50.0
x[B][A] = 0.0
x[B][B] = 0.0
x[B][C] = 0.0
x[B][D] = 50.0
x[C][A] = 110.0
x[C][B] = 0.0
x[C][C] = 0.0
x[C][D] = 90.0
x[D][A] = 0.0
x[D][B] = 100.0
x[D

Warum funktioniert linear Programming / Simplex hier nicht? 

In der Realität liegen hier nicht-lineare Constraints vor:
- die Kostenfunktion ist aufgrund v. Verkehrsaufkommen, unterschiedlicher Dauer der Transportwege nicht linear 
   - ergänzen um Kalman-Filter, dann nicht mehr linear